Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-12 17:03:50--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-12 17:03:50--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   104KB/s    in 2.0s    

2021-10-12 17:03:53 (104 KB/s) - ‘data.zip’ saved [217313/217313]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


# Data Read

In [3]:
import sys
import re
def cleaning(text):
  text= text.lower()
  text=text.strip()
  # text=text.replace('[^\w\s]','')
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
  replace_dot = re.compile("\.")
  text = replace_dot.sub(' ', text)
  replace_q = re.compile("\?")
  text = replace_q.sub(' ', text)
  BAD_SYMBOLS_RE = re.compile('[^0-9a-z +]')
  text = REPLACE_BY_SPACE_RE.sub('' , text)
  text = BAD_SYMBOLS_RE.sub('',text) 
  text= text.strip() 
  return text

In [4]:
def encode(text):
  if text == "F":
    return int(0)
  else: 
    return int(1)

In [5]:
import pandas as pd
df_train_data = pd.read_csv('data/train/train.data.txt', sep ="\t",names = ['word', 'status', 'position','sentence1','sentence2'])
df_train_data['sentence1']=df_train_data['sentence1'].apply(cleaning)
df_train_data['sentence2']=df_train_data['sentence2'].apply(cleaning)

In [6]:
import pandas as pd
df_train_label = pd.read_csv('data/train/train.gold.txt', sep ="\n",names = ['label'])
df_train_label['label'] = df_train_label['label'].apply(encode)
# df_train_label.head(5)

In [7]:
import pandas as pd
df_val = pd.read_csv('data/validation/validation.data.txt', sep ="\t",names = ['word', 'status', 'position','sentence1','sentence2'])
df_val['sentence1']=df_val['sentence1'].apply(cleaning)
df_val['sentence2']=df_val['sentence2'].apply(cleaning)

In [8]:
import pandas as pd
df_val_label = pd.read_csv('data/validation/validation.gold.txt', sep ="\t",names = ['label'])
df_val_label['label'] = df_val_label['label'].apply(encode)
# df_val_label.head(5)

In [9]:
df_raw_train = pd.DataFrame()
df_raw_train['label']= df_train_label['label']
df_raw_train['data'] = df_train_data['sentence1'] +" "+ df_train_data['sentence2']+" "+ df_train_data['word']+" "+ df_train_data['word']
df_raw_train = df_raw_train.reindex(columns=['label', 'data'])
df_raw_train.to_csv('train.csv', index=False)

df_raw_val = pd.DataFrame()
df_raw_val['label']= df_val_label['label']
df_raw_val['data'] = df_val['sentence1'] +" "+ df_val['sentence2']+" "+ df_val['word']+" "+ df_val['word']
df_raw_val = df_raw_val.reindex(columns=['label', 'data'])
df_raw_val.to_csv('valid.csv', index=False)

# Model

In [10]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchtext.legacy import data
import spacy

import logging
from argparse import ArgumentParser

from pdb import set_trace
import dill

In [11]:
#Reproducing same results
SEED = 2021

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True 

In [12]:
def parse_args():
	args = {
		'dataset':"wsd",
		'model':"bilstm",
		'gpu':0,
		'batch_size':256,
		'embedding_dim':300,
		'hidden_dim':128,
		'output_dim':1,
		'num_layers':3,
		'bidirectional':True,
		'dropout':0.2,
		'epochs':5,
		'lr':0.001,
		'combine':"cat",
		'results_dir':"results"
	}
	return check_args(args)	


"""checking arguments"""
def check_args(args):
	# --result_dir
	check_folder(os.path.join(args['results_dir'], args['model'], args['dataset']))

	# --epoch
	try:
			assert args['epochs'] >= 1
	except:
			print('number of epochs must be larger than or equal to one')

	# --batch_size
	try:
			assert args['batch_size'] >= 1
	except:
			print('batch size must be larger than or equal to one')
	return args

def get_device(gpu_no):
	if torch.cuda.is_available():
		torch.cuda.set_device(gpu_no)
		return torch.device('cuda:{}'.format(gpu_no))
	else:
		return torch.device('cpu')

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir

def get_logger(args, phase):
	logging.basicConfig(level=logging.INFO, 
												filename = "{}/{}/{}/{}.log".format(args['results_dir'], args['model'], args['dataset'], phase),
												format = '%(asctime)s - %(message)s', 
												datefmt='%d-%b-%y %H:%M:%S')
	return logging.getLogger(phase)

In [13]:
class bilstm(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        super().__init__()          
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        embedded = self.embedding(text)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        dense_outputs=self.fc(hidden)
        outputs=self.act(dense_outputs)        
        return outputs

In [14]:
## Basic training loop

class Train():
	def __init__(self):
		print("program execution start: {}".format(datetime.datetime.now()))
		self.args = parse_args()
		self.device = get_device(self.args['gpu'])
		self.logger = get_logger(self.args, "train")
		self.logger.info("Arguments: {}".format(self.args))
		self.model_options = parse_args()

    ## TODO: Load your own dataset
		TEXT = data.Field(tokenize='spacy',use_vocab=True,batch_first=True,include_lengths=True)
		LABEL = data.LabelField(use_vocab=False,dtype = torch.float,batch_first=True)
		fields = [('label', LABEL), ('text',TEXT)]
		self.train_data=data.TabularDataset(path = 'train.csv',format = 'csv',fields = fields,skip_header = True)
		self.valid_data=data.TabularDataset(path = 'valid.csv',format = 'csv',fields = fields,skip_header = True)	
		TEXT.build_vocab(self.train_data,self.valid_data,min_freq=1,vectors = "glove.6B.300d")  
		LABEL.build_vocab(self.train_data)
		self.size_of_vocab = len(TEXT.vocab)
		pretrained_embeddings = TEXT.vocab.vectors
		with open('field.pkl', 'wb') as out_strm: 
			dill.dump(TEXT, out_strm)
    ## TODO: Load your own model
		self.model = bilstm(self.size_of_vocab, self.args['embedding_dim'], self.args['hidden_dim'] , self.args['output_dim'], self.args['num_layers'], 
                    self.args['bidirectional'], self.args['dropout'])
		self.model.embedding.weight.data.copy_(pretrained_embeddings)
		self.model.to(self.device)
		self.criterion = nn.BCELoss()
		# self.criterion = nn.CrossEntropyLoss(reduction = 'sum')
		self.criterion = self.criterion.to(self.device)
		self.opt = O.Adam(self.model.parameters(), lr = self.args['lr'])
		self.best_val_acc = None
		self.scheduler = StepLR(self.opt, step_size=5, gamma=0.5)
		self.train_iterator, self.valid_iterator = data.BucketIterator.splits(
    (self.train_data, self.valid_data), 
    batch_size = self.args['batch_size'],
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = self.device)

		print("resource preparation done: {}".format(datetime.datetime.now()))

	def result_checkpoint(self, epoch, train_loss, val_loss, train_acc, val_acc, took):
		if self.best_val_acc is None or val_acc > self.best_val_acc:
			self.best_val_acc = val_acc
			torch.save({
				'accuracy': self.best_val_acc,
				'options': self.model_options,
				'model_dict': self.model.state_dict(),
			}, '{}/{}/{}/best-{}-{}-params.pt'.format(self.args['results_dir'], self.args['model'], self.args['dataset'], self.args['model'], self.args['dataset']))
		self.logger.info('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'
				.format(epoch, train_loss, train_acc, val_loss, val_acc, took))
	
	def train(self):
		self.model.train()
		# self.train_iterator.init_epoch()
		# self.dataset.train_iter.init_epoch()
		n_correct, n_total, n_loss = 0, 0, 0		
		# for batch_idx, batch in enumerate(self.train_iterator):
		for batch in self.train_iterator:
			self.opt.zero_grad()
			text, text_lengths = batch.text
			answer =self.model(text, text_lengths).squeeze() 
			# answer = self.model(batch)
			loss = self.criterion(answer, batch.label)
			acc = binary_accuracy(answer, batch.label)
			# n_correct += torch.max(answer,1)
			# n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
			# n_total += batch.batch_size
			# n_loss += loss.item()
			
			loss.backward()
			torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.5)
			self.opt.step()
		# train_loss = n_loss/n_total
		# train_acc = 100. * n_correct/n_total
		return loss, acc 

	def validate(self):
		self.model.eval()
	#  self.dataset.dev_iter.init_epoch()
		n_correct, n_total, n_loss = 0, 0, 0
		with torch.no_grad():
			for batch in self.valid_iterator:
				text, text_lengths = batch.text
			# for batch_idx, batch in enumerate(self.dataset.dev_iter):
				answer = self.model(text, text_lengths).squeeze()
				loss = self.criterion(answer, batch.label)
				acc = binary_accuracy(answer, batch.label)
				# n_correct += torch.max(answer,1)
				# n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
				# n_total += batch.batch_size
				# n_loss += loss.item()

			# val_loss = n_loss/n_total
			# val_acc = 100. * n_correct/n_total
			return loss, acc

	def execute(self):
		print(" [*] Training starts!")
		print('-' * 99)
		for epoch in range(1, self.args['epochs']+1):
			start = time.time()

			train_loss, train_acc = self.train()
			val_loss, val_acc = self.validate()
			self.scheduler.step()
			
			took = time.time()-start
			self.result_checkpoint(epoch, train_loss, val_loss, train_acc, val_acc, took)

			print('| Epoch {:3d} | train loss {:5.2f} | train acc {:5.2f} | val loss {:5.2f} | val acc {:5.2f} | time: {:5.2f}s |'.format(
				epoch, train_loss, train_acc, val_loss, val_acc, took))
		self.finish()

	def finish(self):
		self.logger.info("[*] Training finished!\n\n")
		print('-' * 99)
		print(" [*] Training finished!")
		print(" [*] Please find the saved model and training log in results_dir")


In [15]:
def binary_accuracy(preds, y):
  rounded_preds = torch.round(preds)    
  correct = (rounded_preds == y).float() 
  acc = correct.sum() / len(correct)
  return acc

In [19]:
## Start training
task = Train()
task.execute()

program execution start: 2021-10-12 17:15:21.115433
resource preparation done: 2021-10-12 17:15:23.381917
 [*] Training starts!
---------------------------------------------------------------------------------------------------
| Epoch   1 | train loss  0.68 | train acc  0.57 | val loss  0.69 | val acc  0.52 | time: 25.03s |
| Epoch   2 | train loss  0.66 | train acc  0.69 | val loss  0.75 | val acc  0.50 | time: 25.14s |
| Epoch   3 | train loss  0.56 | train acc  0.71 | val loss  0.72 | val acc  0.56 | time: 25.24s |
| Epoch   4 | train loss  0.48 | train acc  0.82 | val loss  0.88 | val acc  0.51 | time: 25.37s |
| Epoch   5 | train loss  0.39 | train acc  0.84 | val loss  1.03 | val acc  0.57 | time: 25.38s |
---------------------------------------------------------------------------------------------------
 [*] Training finished!
 [*] Please find the saved model and training log in results_dir


In [17]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2020CSY7657_A_model.zip **

## Upload it to Google drive and ensure that the testing notebook uses the correct link


  adding: data/ (stored 0%)
  adding: data/validation/ (stored 0%)
  adding: data/validation/validation.gold.txt (deflated 87%)
  adding: data/validation/validation.data.txt (deflated 58%)
  adding: data/train/ (stored 0%)
  adding: data/train/train.data.txt (deflated 63%)
  adding: data/train/train.gold.txt (deflated 90%)
  adding: data.zip (stored 0%)
  adding: field.pkl (deflated 11%)
  adding: results/ (stored 0%)
  adding: results/bilstm/ (stored 0%)
  adding: results/bilstm/wsd/ (stored 0%)
  adding: results/bilstm/wsd/best-bilstm-wsd-params.pt (deflated 8%)
  adding: results/bilstm/wsd/train.log (deflated 61%)
  adding: sample_data/ (stored 0%)
  adding: sample_data/anscombe.json (deflated 83%)
  adding: sample_data/README.md (deflated 42%)
  adding: sample_data/california_housing_test.csv (deflated 76%)
  adding: sample_data/california_housing_train.csv (deflated 79%)
  adding: sample_data/mnist_test.csv (deflated 88%)
  adding: sample_data/mnist_train_small.csv (deflated 88%)
